# Constructing a 4x4 Unitary from 2x2 Blocks

This notebook demonstrates how to embed a 2×2 matrix `A` and its complementary block `Ac` into a 4×4 unitary matrix `U`. 
The main steps are:
1. Normalize a random 2×2 matrix `A` so its largest singular value is 1.
2. Compute `Ac = sqrt(I - A^† A)` so that the stacked matrix `[A; Ac]` has orthonormal columns.
3. Complete `[A; Ac]` to a 4×4 unitary by filling with random columns and using QR.
4. Verify unitarity and (optionally) compute a CS decomposition.

In [ ]:
# Numerical linear algebra imports
import numpy as np
from scipy.linalg import sqrtm

# sqrtm computes the matrix square root. We'll use it to form Ac = sqrt(I - A^† A).

## Initialization helper
The `initialisation` function below normalizes a provided 2×2 matrix and returns `A` and its complementary block `Ac`. Comments explain each step.

In [ ]:
def initialisation(x):
    """Normalize a 2x2 matrix x and compute its complementary block Ac.
    Returns:
        A  : normalized 2x2 matrix with largest singular value = 1
        Ac : 2x2 matrix satisfying A^† A + Ac^† Ac = I (within numerical precision)
    """
    # Compute SVD and normalize by largest singular value to keep conditioning reasonable
    U, S, Vt = np.linalg.svd(x)
    largest_singular_value = max(S)
    A = x / largest_singular_value
    # Form Ac so that columns of [A; Ac] are orthonormal: Ac = sqrt(I - A^† A)
    Ac = sqrtm(np.identity(2) - (np.conj(A.T) @ A))
    return A, Ac

## Generate random 2×2 blocks and sanity-check orthogonality
We create two random 2×2 matrices `A_temp` and `B_temp`, initialize them and verify that `A^† A + Ac^† Ac = I` (and similarly for `B`).

In [ ]:
# Create random real matrices (you can switch to complex if needed)
A_temp = np.random.randn(2,2)
B_temp = np.random.randn(2,2)
# Initialize A and B plus their complementary blocks Ac and Bc
A,Ac = initialisation(A_temp)
B,Bc = initialisation(B_temp)

# Quick check: columns of [A; Ac] and [B; Bc] should be orthonormal (I within tol)
print(np.allclose(A.conj().T @ A + Ac.conj().T @ Ac, np.eye(2)), 
      np.allclose(B.conj().T @ B + Bc.conj().T @ Bc, np.eye(2)))

True True


## Complete `[A; Ac]` to a 4×4 unitary via QR
We stack `A` and `Ac` into a 4×2 matrix `V` and then fill the remaining columns with random data. A QR on the full matrix gives orthonormal columns; we extract the complement columns to build `U`.

In [ ]:
from scipy.linalg import sqrtm, qr
# Step 2: Form V = [A; Ac] (4x2)
V = np.vstack([A, Ac])  # Shape: (4, 2)

# Step 3: Generate a random 4x2 matrix to fill out the rest. Use complex entries to avoid accidental collinearity.
Z = np.random.randn(4, 2) + 1j * np.random.randn(4, 2)

# Step 4: QR decomposition of [V | Z] to obtain an orthonormal basis for C^4.
full_matrix = np.hstack([V, Z])  # Shape: (4, 4)
Q, _ = qr(full_matrix)

# Step 5: Extract orthonormal complement columns (the last two columns of Q)
W = Q[:, 2:]  # Shape: (4, 2)

# Step 6: Split W into X (top 2 rows) and Y (bottom 2 rows) to pair with A and Ac.
X = W[0:2, :]
Y = W[2:4, :]

# Step 7: Construct the full 4x4 unitary U by placing the blocks:
# U = [A  X; Ac  Y] where each block is 2x2.
U = np.vstack([np.hstack([A, X]), np.hstack([Ac, Y])])

# Step 8: Verify unitarity numerically (U^† U ≈ I). Use a tight tolerance for small matrices.
print("Is U unitary? →", np.allclose(U.conj().T @ U, np.eye(4), atol=1e-10))

Is U unitary? → True


In [ ]:
# Print some diagnostics about the constructed unitary
print("Shape of U:", U.shape)
print("Is U unitary?", np.allclose(U.conj().T @ U, np.eye(U.shape[0]), atol=1e-10))

Shape of U: (4, 4)
Is U unitary? True


## Optional: CS decomposition
If you want to analyze the block structure further, compute a CS decomposition of `U` with `p=2` to inspect the cosine-sine structure between the two 2-dim subspaces.

In [ ]:
from scipy.linalg import cossin
# Compute a CS decomposition of U partitioned into 2+2 blocks.
Ublock, CS, VblockH = cossin(U, p=2, separate=False)
# Ublock and VblockH are block unitary factors; CS contains cosine and sine values.

array([[ 0.13484613+0.00000000e+00j,  0.99086655+0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.99086655+0.00000000e+00j, -0.13484613+0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        -0.1609811 +4.69097726e-09j,  0.98695749+2.52818688e-10j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        -0.98695749+3.85155896e-08j, -0.1609811 +1.55000369e-09j]])